# Case Study - Automatic Ticket Classification
 


We will be doing topic modelling on the <b>.json</b> data provided by the company. Since this data is not labelled, we need to apply NMF to analyse patterns and classify tickets into the following five clusters based on their products/services:

* Credit card / Prepaid card

* Bank account services

* Theft/Dispute reporting

* Mortgages/loans

* Others 


With the help of topic modelling, we will be able to map each ticket onto its respective department/category. we can then use this data to train any supervised model such as logistic regression, decision tree or random forest. Using this trained model, we can classify any new customer complaint support ticket into its relevant department.

## Importing the necessary libraries

In [1]:
import json 
import numpy as np
import pandas as pd
import re, nltk, spacy, string
import en_core_web_sm
nlp =  en_core_web_sm.load(disable=['parser','ner'])
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from collections import Counter
import swifter
from sklearn.feature_extraction.text import TfidfTransformer

%matplotlib inline

from plotly.offline import plot
import plotly.graph_objects as go
import plotly.express as px

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from pprint import pprint

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings('ignore')

## Loading the data

The data is in JSON format and we need to convert it to a dataframe.

In [2]:
# Opening JSON file 
f = open('complaints-2021-05-14_08_16.json')
 
data = json.load(f)
df=pd.json_normalize(data)

## Data preparation

In [3]:
# Inspect the dataframe to understand the given data.
df.head()

_index      _type      _id  _score   _source.tags  \
0  complaint-public-v2  complaint  3211475     0.0           None   
1  complaint-public-v2  complaint  3229299     0.0  Servicemember   
2  complaint-public-v2  complaint  3199379     0.0           None   
3  complaint-public-v2  complaint  2673060     0.0           None   
4  complaint-public-v2  complaint  3203545     0.0           None   

  _source.zip_code _source.complaint_id                       _source.issue  \
0            90301              3211475   Attempts to collect debt not owed   
1            319XX              3229299     Written notification about debt   
2            77069              3199379  Other features, terms, or problems   
3            48066              2673060      Trouble during payment process   
4            10473              3203545                    Fees or interest   

       _source.date_received _source.state  ... _source.company_response  \
0  2019-04-13T12:00:00-05:00            CA  ...  Closed with explanation   
1  2019-05-01T12:00:00-05:00            GA  ...  Closed with explanation   
2  2019-04-02T12:00:00-05:00            TX  ...  Closed with explanation   
3  2017-09-13T12:00:00-05:00            MI  ...  Closed with explanation   
4  2019-04-05T12:00:00-05:00            NY  ...  Closed with explanation   

        _source.company _source.submitted_via _source.date_sent_to_company  \
0  JPMORGAN CHASE & CO.                   Web    2019-04-13T12:00:00-05:00   
1  JPMORGAN CHASE & CO.                   Web    2019-05-01T12:00:00-05:00   
2  JPMORGAN CHASE & CO.                   Web    2019-04-02T12:00:00-05:00   
3  JPMORGAN CHASE & CO.                   Web    2017-09-14T12:00:00-05:00   
4  JPMORGAN CHASE & CO.              Referral    2019-04-05T12:00:00-05:00   

  _source.company_public_response                         _source.sub_product  \
0                            None                            Credit card debt   
1                            None                            Credit card debt   
2                            None  General-purpose credit card or charge card   
3                            None                  Conventional home mortgage   
4                            None  General-purpose credit card or charge card   

  _source.timely                    _source.complaint_what_happened  \
0            Yes                                                      
1            Yes  Good morning my name is XXXX XXXX and I apprec...   
2            Yes  I upgraded my XXXX XXXX card in XX/XX/2018 and...   
3            Yes                                                      
4            Yes                                                      

                                  _source.sub_issue  \
0                                 Debt is not yours   
1  Didn't receive enough information to verify debt   
2             Problem with rewards from credit card   
3                                              None   
4                         Charged too much interest   

  _source.consumer_consent_provided  
0              Consent not provided  
1                  Consent provided  
2                  Consent provided  
3              Consent not provided  
4                               N/A  

[5 rows x 22 columns]

In [4]:
#print the column names
df.columns

Index(['_index', '_type', '_id', '_score', '_source.tags', '_source.zip_code',
       '_source.complaint_id', '_source.issue', '_source.date_received',
       '_source.state', '_source.consumer_disputed', '_source.product',
       '_source.company_response', '_source.company', '_source.submitted_via',
       '_source.date_sent_to_company', '_source.company_public_response',
       '_source.sub_product', '_source.timely',
       '_source.complaint_what_happened', '_source.sub_issue',
       '_source.consumer_consent_provided'],
      dtype='object')

In [5]:
#Assign new column names
df.rename(columns={'_source.complaint_what_happened':'complaints_what_happened', '_source.product':'tag'}, inplace=True)

In [6]:
df.columns

Index(['_index', '_type', '_id', '_score', '_source.tags', '_source.zip_code',
       '_source.complaint_id', '_source.issue', '_source.date_received',
       '_source.state', '_source.consumer_disputed', 'tag',
       '_source.company_response', '_source.company', '_source.submitted_via',
       '_source.date_sent_to_company', '_source.company_public_response',
       '_source.sub_product', '_source.timely', 'complaints_what_happened',
       '_source.sub_issue', '_source.consumer_consent_provided'],
      dtype='object')

In [7]:
df = df[['complaints_what_happened', 'tag']]

In [8]:
df = df[df['complaints_what_happened'].astype(bool)]
df

complaints_what_happened  \
1      Good morning my name is XXXX XXXX and I apprec...   
2      I upgraded my XXXX XXXX card in XX/XX/2018 and...   
10     Chase Card was reported on XX/XX/2019. However...   
11     On XX/XX/2018, while trying to book a XXXX  XX...   
14     my grand son give me check for {$1600.00} i de...   
...                                                  ...   
78303  After being a Chase Card customer for well ove...   
78309  On Wednesday, XX/XX/XXXX I called Chas, my XXX...   
78310  I am not familiar with XXXX pay and did not un...   
78311  I have had flawless credit for 30 yrs. I've ha...   
78312  Roughly 10+ years ago I closed out my accounts...   

                                                     tag  
1                                        Debt collection  
2                            Credit card or prepaid card  
10     Credit reporting, credit repair services, or o...  
11     Credit reporting, credit repair services, or o...  
14                           Checking or savings account  
...                                                  ...  
78303                        Credit card or prepaid card  
78309                        Credit card or prepaid card  
78310                        Checking or savings account  
78311                        Credit card or prepaid card  
78312                                        Payday loan  

[21072 rows x 2 columns]

## Prepare the text for topic modeling

After removing blank complaints:

* Make the text lowercase
* Remove text in square brackets
* Remove punctuation
* Remove words containing numbers


After all the cleaning operations, performed the following:
* Lemmatize the texts
* Extract the POS tags of the lemmatized text and remove all the words which have tags other than NN[tag == "NN"].


In [9]:
# Function to clean the text and remove all the unnecessary elements.
def clean_data(text):
    text = text.lower() # text to lowercase
    text = re.sub(r'\s\{\$\S*', '',text) # Remove text within curly braces
    text = re.sub(r'\n', '', text) # Remove line breaks
    text = re.sub(r'\(\w*\)', '', text) #remove text within braces
    text = re.sub(r'(\W\s)|(\W$)|(\W\d*)', ' ',text) # Remove punctuation
    text = re.sub(r'x+((/xx)*/\d*\s*)|x*', '',text) #Remove date
    text = re.sub(r'\d+\s', '', text) #Remove other numerical values
    text = re.sub(r' +', ' ',text) #Remove unnecessary white spaces
    return text


In [10]:
# Apply data cleaning to the complaints_what_happened column

df['complaints'] = df['complaints_what_happened'].apply(clean_data)
df.head()

complaints_what_happened  \
1   Good morning my name is XXXX XXXX and I apprec...   
2   I upgraded my XXXX XXXX card in XX/XX/2018 and...   
10  Chase Card was reported on XX/XX/2019. However...   
11  On XX/XX/2018, while trying to book a XXXX  XX...   
14  my grand son give me check for {$1600.00} i de...   

                                                  tag  \
1                                     Debt collection   
2                         Credit card or prepaid card   
10  Credit reporting, credit repair services, or o...   
11  Credit reporting, credit repair services, or o...   
14                        Checking or savings account   

                                           complaints  
1   good morning my name is and i appreciate it if...  
2   i upgraded my card in and was told by the agen...  
10  chase card was reported on however fraudulent ...  
11  on while trying to book a ticket i came across...  
14  my grand son give me check for i deposit it in...

In [11]:
# Function to Lemmatize the texts
def lemmatization(text):
    tokens = word_tokenize(text)
    wordnet_lemmetizer = WordNetLemmatizer()
    lemmatized = [wordnet_lemmetizer.lemmatize(token) for token in tokens]
    lemmatized_str = " ".join(lemmatized)
    return lemmatized_str

In [12]:
#Create a dataframe('df_clean') that will have only the complaints and the lemmatized complaints 
df_clean = pd.DataFrame({'complaints':df['complaints'], 'lemmatized':df['complaints'].apply(lemmatization)})

In [13]:
df_clean.head()

complaints  \
1   good morning my name is and i appreciate it if...   
2   i upgraded my card in and was told by the agen...   
10  chase card was reported on however fraudulent ...   
11  on while trying to book a ticket i came across...   
14  my grand son give me check for i deposit it in...   

                                           lemmatized  
1   good morning my name is and i appreciate it if...  
2   i upgraded my card in and wa told by the agent...  
10  chase card wa reported on however fraudulent a...  
11  on while trying to book a ticket i came across...  
14  my grand son give me check for i deposit it in...

In [14]:
#Write your function to extract the POS tags 

def get_pos_tags(text):
    nn_words = []
    doc = nlp(text)
    for tok in doc:
        if(tok.tag_ == 'NN'):
            nn_words.append(tok.lemma_)
    nn_words_str = " ".join(nn_words)
    return nn_words_str

#this column should contain lemmatized text with all the words removed which have tags other than NN[tag == "NN"].
df_clean["complaint_POS_removed"] =  df_clean.swifter.apply(lambda x: get_pos_tags(x['lemmatized']), axis=1)

Pandas Apply:   0%|          | 0/21072 [00:00<?, ?it/s]

In [15]:
#The clean dataframe should now contain the raw complaint, lemmatized complaint and the complaint after removing POS tags.
df_clean

complaints  \
1      good morning my name is and i appreciate it if...   
2      i upgraded my card in and was told by the agen...   
10     chase card was reported on however fraudulent ...   
11     on while trying to book a ticket i came across...   
14     my grand son give me check for i deposit it in...   
...                                                  ...   
78303  after being a chase card customer for well ove...   
78309  on wednesday i called chas my visa credit card...   
78310  i am not familiar with pay and did not underst...   
78311  i have had flawless credit for yrs i ve had ch...   
78312  roughly years ago i closed out my accounts wit...   

                                              lemmatized  \
1      good morning my name is and i appreciate it if...   
2      i upgraded my card in and wa told by the agent...   
10     chase card wa reported on however fraudulent a...   
11     on while trying to book a ticket i came across...   
14     my grand son give me check for i deposit it in...   
...                                                  ...   
78303  after being a chase card customer for well ove...   
78309  on wednesday i called chas my visa credit card...   
78310  i am not familiar with pay and did not underst...   
78311  i have had flawless credit for yr i ve had cha...   
78312  roughly year ago i closed out my account with ...   

                                   complaint_POS_removed  
1      morning name stop bank service debt verificati...  
2      card agent anniversary date agent information ...  
10     card application identity consent service cred...  
11     ticket offer ticket reward card information of...  
14     son chase account fund chase bank account mone...  
...                                                  ...  
78303  chase card customer decade solicitation credit...  
78309  visa credit card provider claim purchase prote...  
78310  pay risk consumer chase bank app chase year ba...  
78311  credit chase credit card chase freedom problem...  
78312  year account order line credit account payment...  

[21072 rows x 3 columns]

## Exploratory data analysis to get familiar with the data.

Data visulaisation:

*   Visualise the data according to the 'Complaint' character length
*   Using a word cloud find the top 40 words by frequency among all the articles after processing the text
*   Find the top unigrams,bigrams and trigrams by frequency among all the complaints after processing the text. ‘




In [16]:
# Write your code here to visualise the data according to the 'Complaint' character length
plt.figure(figsize=(10,6))
doc_lens = [len(d) for d in df_clean.complaint_POS_removed]
plt.hist(doc_lens, bins = 100)
plt.ylabel('Number of Complaint')
plt.xlabel('Complaint character length')
sns.despine();

#### Find the top 40 words by frequency among all the articles after processing the text.

In [17]:
# Top 40 words frequency wise wordcloud
wordcloud = WordCloud(max_words=40, random_state=1, stopwords=set(STOPWORDS))
wordcloud.generate(str(df_clean['complaint_POS_removed']))
plt.figure(figsize=(12,12))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [18]:
#Removing -PRON- from the text corpus
df_clean['Complaint_clean'] = df_clean['complaint_POS_removed'].str.replace('-PRON-', '')
df_clean

complaints  \
1      good morning my name is and i appreciate it if...   
2      i upgraded my card in and was told by the agen...   
10     chase card was reported on however fraudulent ...   
11     on while trying to book a ticket i came across...   
14     my grand son give me check for i deposit it in...   
...                                                  ...   
78303  after being a chase card customer for well ove...   
78309  on wednesday i called chas my visa credit card...   
78310  i am not familiar with pay and did not underst...   
78311  i have had flawless credit for yrs i ve had ch...   
78312  roughly years ago i closed out my accounts wit...   

                                              lemmatized  \
1      good morning my name is and i appreciate it if...   
2      i upgraded my card in and wa told by the agent...   
10     chase card wa reported on however fraudulent a...   
11     on while trying to book a ticket i came across...   
14     my grand son give me check for i deposit it in...   
...                                                  ...   
78303  after being a chase card customer for well ove...   
78309  on wednesday i called chas my visa credit card...   
78310  i am not familiar with pay and did not underst...   
78311  i have had flawless credit for yr i ve had cha...   
78312  roughly year ago i closed out my account with ...   

                                   complaint_POS_removed  \
1      morning name stop bank service debt verificati...   
2      card agent anniversary date agent information ...   
10     card application identity consent service cred...   
11     ticket offer ticket reward card information of...   
14     son chase account fund chase bank account mone...   
...                                                  ...   
78303  chase card customer decade solicitation credit...   
78309  visa credit card provider claim purchase prote...   
78310  pay risk consumer chase bank app chase year ba...   
78311  credit chase credit card chase freedom problem...   
78312  year account order line credit account payment...   

                                         Complaint_clean  
1      morning name stop bank service debt verificati...  
2      card agent anniversary date agent information ...  
10     card application identity consent service cred...  
11     ticket offer ticket reward card information of...  
14     son chase account fund chase bank account mone...  
...                                                  ...  
78303  chase card customer decade solicitation credit...  
78309  visa credit card provider claim purchase prote...  
78310  pay risk consumer chase bank app chase year ba...  
78311  credit chase credit card chase freedom problem...  
78312  year account order line credit account payment...  

[21072 rows x 4 columns]

#### Find the top unigrams,bigrams and trigrams by frequency among all the complaints after processing the text.

In [19]:
def top_grams(corpus, n=None):
  vec=CountVectorizer(stop_words='english').fit(corpus)
  bag_of_words=vec.transform(corpus)
  sum_words=bag_of_words.sum(axis=0)
  words_freq=[(word,sum_words[0,idx]) for word,idx in vec.vocabulary_.items()]
  words_freq=sorted(words_freq, key=lambda x: x[1], reverse=True )
  return words_freq[:n]

common_words = top_grams(df_clean['Complaint_clean'].values.astype('U'),30)
df_unigram = pd.DataFrame(common_words,columns=['unigram', 'count'])

fig = go.Figure([go.Bar(x=df_unigram['unigram'], y=df_unigram['count'])])
fig.update_layout(title=go.layout.Title(text='Top 30 unigrams in the cleaned complaint text')) 
fig.show()
    
    

In [20]:
#Print the top 10 words in the unigram frequency
df_unigram.head(10)

unigram  count
0    chase  54491
1  account  47556
2   credit  32787
3     card  30436
4     bank  21568
5  payment  21367
6     time  16311
7      day  13322
8   charge  12082
9    money  12001

In [21]:
#Write your code here to find the top 30 bigram frequency among the complaints in the cleaned datafram(df_clean). 
def top_grams(corpus, n=None):
    vec=CountVectorizer(stop_words='english',ngram_range=(2, 2)).fit(corpus)
    bag_of_words=vec.transform(corpus)
    sum_words=bag_of_words.sum(axis=0)
    words_freq=[(word,sum_words[0,idx]) for word,idx in vec.vocabulary_.items()]
    words_freq=sorted(words_freq, key=lambda x: x[1], reverse=True )
    return words_freq[:n]

common_words = top_grams(df_clean['Complaint_clean'].values.astype('U'),30)
df_bigram = pd.DataFrame(common_words,columns=['bigram', 'count'])

fig = go.Figure([go.Bar(x=df_bigram['bigram'], y=df_bigram['count'])])
fig.update_layout(title=go.layout.Title(text='Top 30 bigram in the cleaned complaint text')) 
fig.show()

In [22]:
#Print the top 10 words in the bigram frequency
df_bigram.head(10)

bigram  count
0       credit card  12931
1     credit report   3892
2     account chase   3452
3     chase account   3149
4      chase credit   3097
5  customer service   3082
6   account account   3041
7      bank account   2639
8        chase bank   2244
9        debit card   2067

In [23]:
#Write your code here to find the top 30 trigram frequency among the complaints in the cleaned datafram(df_clean). 
def top_grams(corpus,n=None):
  vec=CountVectorizer(stop_words='english',ngram_range=(3, 3)).fit(corpus)
  bag_of_words=vec.transform(corpus)
  sum_words=bag_of_words.sum(axis=0)
  words_freq=[(word, sum_words[0,idx]) for word,idx in vec.vocabulary_.items()]
  words_freq=sorted(words_freq, key= lambda x: x[1], reverse=True)
  return words_freq[:n]

common_words=top_grams(df_clean['Complaint_clean'].values.astype('U'), 30)
df_trigram=pd.DataFrame(common_words, columns=['trigram', 'count'])

fig=go.Figure([go.Bar(x=df_trigram['trigram'], y=df_trigram['count'])])
fig.update_layout(title=go.layout.Title(text='Top 30 trigram in the cleaned complaint text')) 
fig.show()

In [24]:
#Print the top 10 words in the trigram frequency
df_trigram.head(10)

trigram  count
0       chase credit card   2027
1     credit card account   1156
2     credit card company   1005
3       credit card chase    846
4      credit card credit    589
5   inquiry credit report    560
6     account credit card    488
7        card credit card    483
8  chase checking account    394
9    credit report credit    389

In [25]:
df_clean

complaints  \
1      good morning my name is and i appreciate it if...   
2      i upgraded my card in and was told by the agen...   
10     chase card was reported on however fraudulent ...   
11     on while trying to book a ticket i came across...   
14     my grand son give me check for i deposit it in...   
...                                                  ...   
78303  after being a chase card customer for well ove...   
78309  on wednesday i called chas my visa credit card...   
78310  i am not familiar with pay and did not underst...   
78311  i have had flawless credit for yrs i ve had ch...   
78312  roughly years ago i closed out my accounts wit...   

                                              lemmatized  \
1      good morning my name is and i appreciate it if...   
2      i upgraded my card in and wa told by the agent...   
10     chase card wa reported on however fraudulent a...   
11     on while trying to book a ticket i came across...   
14     my grand son give me check for i deposit it in...   
...                                                  ...   
78303  after being a chase card customer for well ove...   
78309  on wednesday i called chas my visa credit card...   
78310  i am not familiar with pay and did not underst...   
78311  i have had flawless credit for yr i ve had cha...   
78312  roughly year ago i closed out my account with ...   

                                   complaint_POS_removed  \
1      morning name stop bank service debt verificati...   
2      card agent anniversary date agent information ...   
10     card application identity consent service cred...   
11     ticket offer ticket reward card information of...   
14     son chase account fund chase bank account mone...   
...                                                  ...   
78303  chase card customer decade solicitation credit...   
78309  visa credit card provider claim purchase prote...   
78310  pay risk consumer chase bank app chase year ba...   
78311  credit chase credit card chase freedom problem...   
78312  year account order line credit account payment...   

                                         Complaint_clean  
1      morning name stop bank service debt verificati...  
2      card agent anniversary date agent information ...  
10     card application identity consent service cred...  
11     ticket offer ticket reward card information of...  
14     son chase account fund chase bank account mone...  
...                                                  ...  
78303  chase card customer decade solicitation credit...  
78309  visa credit card provider claim purchase prote...  
78310  pay risk consumer chase bank app chase year ba...  
78311  credit chase credit card chase freedom problem...  
78312  year account order line credit account payment...  

[21072 rows x 4 columns]

## Feature Extraction
Convert the raw texts to a matrix of TF-IDF features

**max_df** is used for removing terms that appear too frequently, also known as "corpus-specific stop words"
max_df = 0.95 means "ignore terms that appear in more than 95% of the complaints"

**min_df** is used for removing terms that appear too infrequently
min_df = 2 means "ignore terms that appear in less than 2 complaints"

In [26]:
#Write your code here to initialise the TfidfVectorizer 
tfidf = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')

#### Create a document term matrix using fit_transform

The contents of a document term matrix are tuples of (complaint_id,token_id) tf-idf score:
The tuples that are not there have a tf-idf score of 0

In [27]:
#Write your code here to create the Document Term Matrix by transforming the complaints column present in df_clean.
dtm = tfidf.fit_transform(df_clean['complaints'])

## Topic Modelling using NMF

Non-Negative Matrix Factorization (NMF) is an unsupervised technique so there are no labeling of topics that the model will be trained on. The way it works is that, NMF decomposes (or factorizes) high-dimensional vectors into a lower-dimensional representation. These lower-dimensional vectors are non-negative which also means their coefficients are non-negative.

In this task we will perform the following:

* Find the best number of clusters 
* Apply the best number to create word clusters
* Inspect & validate the correction of each cluster wrt the complaints 
* Correct the labels if needed 
* Map the clusters to topics/cluster names

In [28]:
from sklearn.decomposition import NMF

## Manual Topic Modeling
We need to do take the trial & error approach to find the best num of topics for your NMF model.

The only parameter that is required is the number of components i.e. the number of topics we want. This is the most crucial step in the whole topic modeling process and will greatly affect how good our final topics are.

In [29]:
#Load your nmf_model with the n_components i.e 5
num_topics = 5

#keep the random_state =40
nmf_model = NMF(random_state=40, n_components=num_topics)

In [30]:
nmf_model.fit(dtm)
len(tfidf.get_feature_names_out())

14985

In [31]:
#Print the Top15 words for each of the topics
words = np.array(tfidf.get_feature_names_out())
topic_words_df = pd.DataFrame(np.zeros((num_topics, 15)), index=[f'Topic {i + 1}' for i in range(num_topics)],
                           columns=[f'Word {i + 1}' for i in range(15)]).astype(str)

for i in range(num_topics):
    ix = nmf_model.components_[i].argsort()[::-1][:15]
    topic_words_df.iloc[i] = words[ix]

topic_words_df

Word 1    Word 2    Word 3   Word 4        Word 5     Word 6  \
Topic 1     card     chase   charges   charge       dispute      fraud   
Topic 2   credit      card    report  inquiry         chase  inquiries   
Topic 3     loan  mortgage     chase     home  modification   property   
Topic 4  account      bank     check    chase         money      funds   
Topic 5  payment      late  payments  balance          paid        pay   

              Word 7    Word 8     Word 9    Word 10   Word 11   Word 12  \
Topic 1       called  received        did       told  merchant     claim   
Topic 2         hard     cards  reporting      score  accounts   account   
Topic 3  foreclosure  payments      house       bank     years      sale   
Topic 4     checking    closed    deposit     branch      told  accounts   
Topic 5        month      fees        fee  statement      auto   account   

             Word 13  Word 14   Word 15  
Topic 1  transaction     said    refund  
Topic 2       closed     debt   removed  
Topic 3       morgan     rate        jp  
Topic 4    deposited     said  business  
Topic 5        chase  charged   monthly

In [32]:
#Create the best topic for each complaint in terms of integer value 0,1,2,3 & 4
topic_results = nmf_model.transform(dtm)
topic_results.argmax(axis=1)

array([2, 0, 1, ..., 0, 4, 4], dtype=int64)

In [33]:
#Assign the best topic to each of the cmplaints in Topic Column

df_clean['Topic'] = topic_results.argmax(axis=1)

In [34]:
df_clean.head()

complaints  \
1   good morning my name is and i appreciate it if...   
2   i upgraded my card in and was told by the agen...   
10  chase card was reported on however fraudulent ...   
11  on while trying to book a ticket i came across...   
14  my grand son give me check for i deposit it in...   

                                           lemmatized  \
1   good morning my name is and i appreciate it if...   
2   i upgraded my card in and wa told by the agent...   
10  chase card wa reported on however fraudulent a...   
11  on while trying to book a ticket i came across...   
14  my grand son give me check for i deposit it in...   

                                complaint_POS_removed  \
1   morning name stop bank service debt verificati...   
2   card agent anniversary date agent information ...   
10  card application identity consent service cred...   
11  ticket offer ticket reward card information of...   
14  son chase account fund chase bank account mone...   

                                      Complaint_clean  Topic  
1   morning name stop bank service debt verificati...      2  
2   card agent anniversary date agent information ...      0  
10  card application identity consent service cred...      1  
11  ticket offer ticket reward card information of...      1  
14  son chase account fund chase bank account mone...      3

In [35]:
# Print the first 5 Complaint for each of the Topics
First5_comp=df_clean.groupby('Topic').head(5)
First5_comp.sort_values('Topic')

complaints  \
2    i upgraded my card in and was told by the agen...   
21   on i made a payment to an online retailer usin...   
43   chase sent me an email today with the innocuou...   
40    a sofa love seat table and chairs and was nev...   
32   my chase amazon card was declined for a cateri...   
10   chase card was reported on however fraudulent ...   
11   on while trying to book a ticket i came across...   
15                       can you please remove inquiry   
23   i have a chase credit card which is incorrectl...   
26   i have reached out to several times in attempt...   
1    good morning my name is and i appreciate it if...   
81   i bought a new on from in i paid the loan in b...   
73   i have wired through one of chase branches in ...   
57   i have been trying to do a loan modification w...   
99   my name is i have missed a payment in and was ...   
27   i opened an account with chase bank on and use...   
24      mishandling of this account by chase auto and    
17   with out notice j p morgan chase restricted my...   
14   my grand son give me check for i deposit it in...   
35   i opened the saving account for the bonus i wa...   
30   to whom it may concern chase bank charged wron...   
20   during the summer months i eperience a decline...   
58   i made a purchase of on i made payments of on ...   
82   i recently called to ask chase bank why they r...   
115  multiple times with the latest being chase has...   

                                            lemmatized  \
2    i upgraded my card in and wa told by the agent...   
21   on i made a payment to an online retailer usin...   
43   chase sent me an email today with the innocuou...   
40   a sofa love seat table and chair and wa never ...   
32   my chase amazon card wa declined for a caterin...   
10   chase card wa reported on however fraudulent a...   
11   on while trying to book a ticket i came across...   
15                       can you please remove inquiry   
23   i have a chase credit card which is incorrectl...   
26   i have reached out to several time in attempt ...   
1    good morning my name is and i appreciate it if...   
81   i bought a new on from in i paid the loan in b...   
73   i have wired through one of chase branch in th...   
57   i have been trying to do a loan modification w...   
99   my name is i have missed a payment in and wa d...   
27   i opened an account with chase bank on and use...   
24       mishandling of this account by chase auto and   
17   with out notice j p morgan chase restricted my...   
14   my grand son give me check for i deposit it in...   
35   i opened the saving account for the bonus i wa...   
30   to whom it may concern chase bank charged wron...   
20   during the summer month i eperience a decline ...   
58   i made a purchase of on i made payment of on a...   
82   i recently called to ask chase bank why they r...   
115  multiple time with the latest being chase ha m...   

                                 complaint_POS_removed  \
2    card agent anniversary date agent information ...   
21   payment retailer chase pay chase website scam ...   
43   chase email today title information change cha...   
40   sofa love seat table chair debit card bank vis...   
32   chase catering order brother funeral amount ch...   
10   card application identity consent service cred...   
11   ticket offer ticket reward card information of...   
15                                             inquiry   
23             credit card credit report company issue   
26   time attempt inquiry creditor inquiry report a...   
1    morning name stop bank service debt verificati...   
81   loan title chase time resolution time nothing ...   
73   chase branch eact reinstatement amount order f...   
57       loan modification chase runaround payment doc   
99   name payment day past time loan modification l...   
27   account code bonus term everything account dep...   
24                                  acco

#### After evaluating the mapping, if the topics assigned are correct then assign these names to the relevant topic:
* Bank Account services
* Credit card or prepaid card
* Theft/Dispute Reporting
* Mortgage/Loan
* Others

In [36]:
# Create the dictionary of Topic names and Topics
Topic_names = {0:'Account Services', 1:'Others', 2:'Mortgage/Loan', 3:'Credit card or prepaid card', 4:'Theft/Dispute Reporting'}

# Replace Topics with Topic Names
df_clean['Topic'] = df_clean['Topic'].map(Topic_names)

In [37]:
df_clean

complaints  \
1      good morning my name is and i appreciate it if...   
2      i upgraded my card in and was told by the agen...   
10     chase card was reported on however fraudulent ...   
11     on while trying to book a ticket i came across...   
14     my grand son give me check for i deposit it in...   
...                                                  ...   
78303  after being a chase card customer for well ove...   
78309  on wednesday i called chas my visa credit card...   
78310  i am not familiar with pay and did not underst...   
78311  i have had flawless credit for yrs i ve had ch...   
78312  roughly years ago i closed out my accounts wit...   

                                              lemmatized  \
1      good morning my name is and i appreciate it if...   
2      i upgraded my card in and wa told by the agent...   
10     chase card wa reported on however fraudulent a...   
11     on while trying to book a ticket i came across...   
14     my grand son give me check for i deposit it in...   
...                                                  ...   
78303  after being a chase card customer for well ove...   
78309  on wednesday i called chas my visa credit card...   
78310  i am not familiar with pay and did not underst...   
78311  i have had flawless credit for yr i ve had cha...   
78312  roughly year ago i closed out my account with ...   

                                   complaint_POS_removed  \
1      morning name stop bank service debt verificati...   
2      card agent anniversary date agent information ...   
10     card application identity consent service cred...   
11     ticket offer ticket reward card information of...   
14     son chase account fund chase bank account mone...   
...                                                  ...   
78303  chase card customer decade solicitation credit...   
78309  visa credit card provider claim purchase prote...   
78310  pay risk consumer chase bank app chase year ba...   
78311  credit chase credit card chase freedom problem...   
78312  year account order line credit account payment...   

                                         Complaint_clean  \
1      morning name stop bank service debt verificati...   
2      card agent anniversary date agent information ...   
10     card application identity consent service cred...   
11     ticket offer ticket reward card information of...   
14     son chase account fund chase bank account mone...   
...                                                  ...   
78303  chase card customer decade solicitation credit...   
78309  visa credit card provider claim purchase prote...   
78310  pay risk consumer chase bank app chase year ba...   
78311  credit chase credit card chase freedom problem...   
78312  year account order line credit account payment...   

                             Topic  
1                    Mortgage/Loan  
2                 Account Services  
10                          Others  
11                          Others  
14     Credit card or prepaid card  
...                            ...  
78303                       Others  
78309             Account Services  
78310             Account Services  
78311      Theft/Dispute Reporting  
78312      Theft/Dispute Reporting  

[21072 rows x 5 columns]

## Supervised model to predict any new complaints to the relevant Topics.

We have built a model to create the topics for each complaints.Now in the below section we will use them to classify any new complaints.

*Since we will be using supervised learning technique we have to convert the topic names to numbers(numpy arrays only understand numbers)*

In [38]:
# Create the dictionary again of Topic names and Topics

Topic_names = {'Account Services':0, 'Others':1, 'Mortgage/Loan':2, 'Credit card or prepaid card':3, 'Theft/Dispute Reporting':4}
# Replace Topics with Topic Names
df_clean['Topic'] = df_clean['Topic'].map(Topic_names)

In [39]:
df_clean

complaints  \
1      good morning my name is and i appreciate it if...   
2      i upgraded my card in and was told by the agen...   
10     chase card was reported on however fraudulent ...   
11     on while trying to book a ticket i came across...   
14     my grand son give me check for i deposit it in...   
...                                                  ...   
78303  after being a chase card customer for well ove...   
78309  on wednesday i called chas my visa credit card...   
78310  i am not familiar with pay and did not underst...   
78311  i have had flawless credit for yrs i ve had ch...   
78312  roughly years ago i closed out my accounts wit...   

                                              lemmatized  \
1      good morning my name is and i appreciate it if...   
2      i upgraded my card in and wa told by the agent...   
10     chase card wa reported on however fraudulent a...   
11     on while trying to book a ticket i came across...   
14     my grand son give me check for i deposit it in...   
...                                                  ...   
78303  after being a chase card customer for well ove...   
78309  on wednesday i called chas my visa credit card...   
78310  i am not familiar with pay and did not underst...   
78311  i have had flawless credit for yr i ve had cha...   
78312  roughly year ago i closed out my account with ...   

                                   complaint_POS_removed  \
1      morning name stop bank service debt verificati...   
2      card agent anniversary date agent information ...   
10     card application identity consent service cred...   
11     ticket offer ticket reward card information of...   
14     son chase account fund chase bank account mone...   
...                                                  ...   
78303  chase card customer decade solicitation credit...   
78309  visa credit card provider claim purchase prote...   
78310  pay risk consumer chase bank app chase year ba...   
78311  credit chase credit card chase freedom problem...   
78312  year account order line credit account payment...   

                                         Complaint_clean  Topic  
1      morning name stop bank service debt verificati...      2  
2      card agent anniversary date agent information ...      0  
10     card application identity consent service cred...      1  
11     ticket offer ticket reward card information of...      1  
14     son chase account fund chase bank account mone...      3  
...                                                  ...    ...  
78303  chase card customer decade solicitation credit...      1  
78309  visa credit card provider claim purchase prote...      0  
78310  pay risk consumer chase bank app chase year ba...      0  
78311  credit chase credit card chase freedom problem...      4  
78312  year account order line credit account payment...      4  

[21072 rows x 5 columns]

In [40]:
# Keep the columns"complaint_what_happened" & "Topic" only in the new dataframe --> training_data
training_data = df_clean.drop(['lemmatized', 'complaint_POS_removed', 'Complaint_clean'], axis=1)

In [41]:
training_data

complaints  Topic
1      good morning my name is and i appreciate it if...      2
2      i upgraded my card in and was told by the agen...      0
10     chase card was reported on however fraudulent ...      1
11     on while trying to book a ticket i came across...      1
14     my grand son give me check for i deposit it in...      3
...                                                  ...    ...
78303  after being a chase card customer for well ove...      1
78309  on wednesday i called chas my visa credit card...      0
78310  i am not familiar with pay and did not underst...      0
78311  i have had flawless credit for yrs i ve had ch...      4
78312  roughly years ago i closed out my accounts wit...      4

[21072 rows x 2 columns]

#### Applying the supervised models on the training data created. In this process, we are going to do the following:
* Create the vector counts using Count Vectoriser
* Transform the word vecotr to tf-idf
* Create the train & test data using the train_test_split on the tf-idf & topics


In [42]:
# Write your code to get the Vector count
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(training_data.complaints)

# Write your code here to transform the word vector to tf-idf
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

You have to try atleast 3 models on the train & test data from these options:
* Logistic regression
* Decision Tree
* Random Forest
* Naive Bayes (optional)

**Using the required evaluation metrics judge the tried models and select the ones performing the best**

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import classification_report

In [44]:
# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data.Topic, test_size=0.2, random_state=42)

**1. Logistic Regression**

In [45]:
# Logistic Regression
lr = LogisticRegression().fit(X_train, y_train)
predicted = lr.predict(X_test)

print(classification_report(y_true=y_test, y_pred=predicted))

              precision    recall  f1-score   support

           0       0.93      0.96      0.95       944
           1       0.96      0.95      0.96       914
           2       0.98      0.95      0.96       714
           3       0.94      0.98      0.96      1123
           4       0.97      0.90      0.93       520

    accuracy                           0.95      4215
   macro avg       0.96      0.95      0.95      4215
weighted avg       0.95      0.95      0.95      4215



**2. Decision Tree**

In [46]:
# Decision tree classifier
dt = DecisionTreeClassifier().fit(X_train, y_train)
predicted = dt.predict(X_test)

print(classification_report(y_pred=predicted, y_true=y_test))

              precision    recall  f1-score   support

           0       0.75      0.72      0.73       944
           1       0.80      0.80      0.80       914
           2       0.77      0.81      0.79       714
           3       0.79      0.80      0.80      1123
           4       0.70      0.67      0.69       520

    accuracy                           0.77      4215
   macro avg       0.76      0.76      0.76      4215
weighted avg       0.77      0.77      0.77      4215



**3. Random Forest Classifier**

In [47]:
rfc = RandomForestClassifier(max_depth=10)
rfc.fit(X_train, y_train)
predicted = rfc.predict(X_test)

print(classification_report(y_pred=predicted, y_true=y_test))

              precision    recall  f1-score   support

           0       0.83      0.68      0.74       944
           1       0.78      0.80      0.79       914
           2       0.88      0.78      0.82       714
           3       0.60      0.97      0.74      1123
           4       1.00      0.16      0.28       520

    accuracy                           0.73      4215
   macro avg       0.82      0.68      0.68      4215
weighted avg       0.79      0.73      0.71      4215



**4. Gaussian Naive Bayes**

In [48]:
nb = GaussianNB().fit(X_train.toarray(), y_train)
predicted = nb.predict(X_test.toarray())

print(classification_report(y_pred=predicted, y_true=y_test))

              precision    recall  f1-score   support

           0       0.48      0.35      0.40       944
           1       0.35      0.27      0.30       914
           2       0.52      0.50      0.51       714
           3       0.46      0.30      0.37      1123
           4       0.18      0.51      0.27       520

    accuracy                           0.36      4215
   macro avg       0.40      0.38      0.37      4215
weighted avg       0.42      0.36      0.37      4215



**Clearly Logistic Regression is performing better**

### Infering the best model

In [49]:
# Some sample complaints to infer model

df_complaints = pd.DataFrame({'complaints': ["I can not get from chase who services my mortgage, who owns it and who has original loan docs", 
                                  "The bill amount of my credit card was debited twice. Please look into the matter and resolve at the earliest.",
                                  "I want to open a salary account at your downtown branch. Please provide me the procedure.",
                                  "Yesterday, I received a fraudulent email regarding renewal of my services.",
                                  "What is the procedure to know my CIBIL score?",
                                  "I need to know the number of bank branches and their locations in the city of Dubai"]})
df_complaints

complaints
0  I can not get from chase who services my mortg...
1  The bill amount of my credit card was debited ...
2  I want to open a salary account at your downto...
3  Yesterday, I received a fraudulent email regar...
4      What is the procedure to know my CIBIL score?
5  I need to know the number of bank branches and...

In [50]:
def predict_lr(text):
    Topic_names = {0:'Account Services', 1:'Others', 2:'Mortgage/Loan', 3:'Credit card or prepaid card', 4:'Theft/Dispute Reporting'}
    X_new_counts = count_vect.transform(text)
    X_new_tfidf = tfidf_transformer.transform(X_new_counts)
    predicted = lr.predict(X_new_tfidf)
    return Topic_names[predicted[0]]

In [51]:
df_complaints['tag'] = df_complaints['complaints'].apply(lambda x: predict_lr([x]))
df_complaints

complaints  \
0  I can not get from chase who services my mortg...   
1  The bill amount of my credit card was debited ...   
2  I want to open a salary account at your downto...   
3  Yesterday, I received a fraudulent email regar...   
4      What is the procedure to know my CIBIL score?   
5  I need to know the number of bank branches and...   

                           tag  
0                Mortgage/Loan  
1                       Others  
2  Credit card or prepaid card  
3             Account Services  
4                Mortgage/Loan  
5  Credit card or prepaid card

## Conclusion
    
* **As expected 5 topics were indetified namely:**
    1. Account Services
    2. Others
    3. Mortgage/Loan
    4. Credit card or prepaid card
    5. Theft/Dispute Reporting
    
    
* **Tried 4 models on the data with accuracies as follows:**

    | Model       | Accuracy |
    | ----------- | ----------- |
    | Logistic Regression      | 0.95       |
    | Decision Tree   | 0.77        |
    | Random Forest      | 0.73       |
    | Naive Bayes   | 0.36        |
    
 <br /> 
* **Logistic Regression has highest accuracy of `0.95`, Hence is a good fit.**